In [1]:
pip install firebase-admin

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 12.7/12.7 MB 66.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 94.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 4.4/4.4 MB 88.1 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.w

In [2]:
pip install flask-sqlalchemy mysql-connector-python

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ----------------------------- ---------- 10.5/14.3 MB 54.4 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 49.9 MB/s eta 0:00:00
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 39.8 MB/s eta 0:00:00
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
Note: you may need to restart the kernel to

In [29]:
pip show firebase_admin

Name: firebase-admin
Version: 6.6.0
Summary: Firebase Admin Python SDK
Home-page: https://firebase.google.com/docs/admin/setup/
Author: Firebase
Author-email: 
License: Apache License 2.0
Location: c:\users\smhrd\anaconda3\envs\bareumi\lib\site-packages
Requires: cachecontrol, google-api-core, google-api-python-client, google-cloud-firestore, google-cloud-storage, pyjwt
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install flask-cors

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install flask-socketio

  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install flask-sock

In [19]:
import firebase_admin
from firebase_admin import credentials, db  # Realtime Database 관련 모듈

from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from flask_sock import Sock 
from datetime import datetime

import json
# 앱 초기화
app = Flask(__name__)
sock = Sock(app)
CORS(app,
     resources = {r"/sensor/*": {"origins": ["http://localhost:3000"]}},
     supports_credentials=True
     )

# Firebase 초기화
if not firebase_admin._apps:
    cred = credentials.Certificate("./key/serviceKey.json")  # Firebase 서비스 계정 키 파일의 경로
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://bareu-2fa47-default-rtdb.firebaseio.com/'  
    })


sensor_set = {"status": "OFF"}
clients = []

# # Firebase 데이터경로
# db_ref = db.reference('sensor_data') # 데이터 저장 및 읽기 경로

@app.get("/")
def index():
     return render_template('index.html')
 
@app.get("/send-data")
def to_input():
    return render_template('testInput.html')

@app.post("/send-data")
def send_firebase():

    data = request.form.to_dict()
    custom_key = f"test_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}"
    print(custom_key)
    db_ref = db.reference("test")

    db_ref.child(custom_key).set({"value": data["test"]})

    return jsonify({
        "message": "send data success",
        "firebase_key": custom_key,
        "data": data
    }),200


def sensor_status_update(data):
    for client in clients:
        try:
            client.send(data)
        except:
            clients.remove(client)

@app.get("/sensor/on")
def sensor_on() :
    print("센서연결")
    sensor_set["status"] = "ON"
    print(sensor_set["status"])
    sensor_status_update(sensor_set["status"])
    print(f"Value updated to: {sensor_set}")
    return jsonify({"message":"센서연결 완료!","sensorSet": True}),200

@app.get("/sensor/pause")
def sensor_pause() :
    print("센서 기록정지")
    sensor_set["status"] = "OFF"
    print(sensor_set["status"])
    sensor_status_update(sensor_set["status"])
    print(f"Value updated to: {sensor_set}")
    return jsonify({"message":"기록정지 완료!","sensorSet": False}),200

@app.get("/sensor/off")
def sensor_off() :
    print("센서 종료")
    sensor_set["status"] = "OFF"
    print(sensor_set["status"])
    sensor_status_update(sensor_set["status"])
    print(f"Value updated to: {sensor_set}")
    return jsonify({"message":"중지 완료!","sensorSet": False}),200

@sock.route('/ws')
def websocket(ws):
    print(f"Client Connected! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    clients.append(ws)
    try:
        while True:
            message = ws.receive()
            print("Received from ESP32:", message)
            # ESP32가 전송한 데이터를 출력
            for client in clients:
                client.send("Data received!")
    except Exception as e:
        print(f"Client Disconnected: {e}")
        clients.remove(ws)

    

if __name__ =="__main__":
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader = False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.56:5000
Press CTRL+C to quit
192.168.219.56 - - [16/Dec/2024 16:04:19] "GET /sensor/on HTTP/1.1" 200 -


센서연결
ON
Value updated to: {'status': 'ON'}


192.168.219.56 - - [16/Dec/2024 16:04:25] "GET /sensor/pause HTTP/1.1" 200 -


센서 기록정지
OFF
Value updated to: {'status': 'OFF'}


...

센서 기록정지
OFF
Value updated to: {'status': 'OFF'}


192.168.219.56 - - [16/Dec/2024 16:19:37] "GET /sensor/off HTTP/1.1" 200 -


센서 종료
OFF
Value updated to: {'status': 'OFF'}


192.168.219.47 - - [16/Dec/2024 16:57:18] "GET /ws HTTP/1.1" 200 -


Client Connected! 2024-12-16 16:57:18
Received from ESP32: Hello from ESP32!
Client Disconnected: Connection closed: 1005 
